### <center>Indian Institute of Management Ahmedabad</center>
#### <center>Machine Learning with Big Data: Assignment 2</center>
<center>ePGD ABA 2021-22</center>

<font color = "purple"><center>Submitted by : <br>
    Group 8 <br>
    Monica Yadav <br>
    Shubhanjali Sharma <br>
    Shivam Singh</center></font>

<b> Question 14.5 Collaborative Filtering </b>

In [4]:
from pathlib import Path

import heapq
from collections import defaultdict
from scipy.stats import pearsonr
import pandas as pd
import matplotlib.pylab as plt
import numpy as np

from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split

from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity

<font color='purple'> Loading Data </font>

In [5]:
ratings = pd.DataFrame([
    ['LN','SQL',4], ['LN','Python',3], ['LN','Forecast',2], ['LN','R Prog',4], ['LN','Regression',2], 
    ['MH','SQL',3], ['MH','Spatial',4], ['MH','Python',4], 
    ['JH','SQL',2],['JH','Spatial',2],
    ['EN','SQL',4],['EN','DM in R',4],['EN','R Prog',4],['EN','Regression',3],
    ['DU','SQL',4],['DU','Spatial',4],
    ['FL','Spatial',4],
    ['GL','Spatial',4],
    ['AH','Spatial',3],
    ['SA', 'PA 1',4],
    ['RW','PA 1',2],['RW','Hadoop',4],
    ['BA','PA 1',4],
    ['MG','PA 1',4], ['MG','Forecast',4],
    ['AF','PA 1',4],
    ['KG','PA 1',3],
    ['DS','SQL',4],['DS','DM in R',2],['DS','R Prog',4],], 
    columns=['User', 'Course', 'Rating'])


In [6]:
subject_rating = ratings.pivot_table(index='User', columns='Course', values='Rating')
df = subject_rating
df

Course,DM in R,Forecast,Hadoop,PA 1,Python,R Prog,Regression,SQL,Spatial
User,,,,,,,,,
AF,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN
AH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
BA,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN
DS,2.0,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN
DU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0
EN,4.0,NaN,NaN,NaN,NaN,4.0,3.0,4.0,NaN
FL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
GL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
JH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0


### Part A </font>

<font color='purple'>Imputing the blanks with mean ratings given by the user as a placeholder to calculate correlations </font>

In [13]:
ratings_impute=df.apply(lambda row: row.fillna(row.mean()), axis=1)
ratings_impute

Course,DM in R,Forecast,Hadoop,PA 1,Python,R Prog,Regression,SQL,Spatial
User,,,,,,,,,
AF,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.0,4.000000
AH,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.0,3.000000
BA,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.0,4.000000
DS,2.000000,3.333333,3.333333,3.333333,3.333333,4.000000,3.333333,4.0,3.333333
DU,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.0,4.000000
EN,4.000000,3.750000,3.750000,3.750000,3.750000,4.000000,3.000000,4.0,3.750000
FL,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.0,4.000000
GL,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.0,4.000000
JH,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.0,2.000000


<font color='purple'> Correlation will be computed only for users who have rated more than
one course, also provided those ratings differ from each other. For eg: If
user1 gave a rating of 3 to course1 and 3 to course2, the correlation will not be computed in this case as the mean will be equal to individual ratings, making correlation NaN. 
    
    
   For EN, correlation will be computed with users who have co-rated similar courses as EN. For example, EN has rated SQL, DM in R, R prog, Regression. Other users who have rated all these courses are: LN, MH, JH, DU, DS. Out of these, JH has only given a rating of 2 throughout and DU has only given a rating of 4 throughout so correlation will not be computed for JH and DU. Therefore EN will have correlation only with LN,MH and DS. 
   
   (We have replaced NaNs with "-" in the below matrix). </font>

In [20]:
ratings_impute.T.corr().fillna("-")

User,AF,AH,BA,DS,DU,EN,FL,GL,JH,KG,LN,MG,MH,RW,SA
User,,,,,,,,,,,,,,,
AF,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
AH,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
BA,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
DS,-,-,-,1,-,-7.85046e-17,-,-,-,-,0.408248,-,-0.333333,0,-
DU,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
EN,-,-,-,-7.85046e-17,-,1,-,-,-,-,0.721688,-,-0.235702,0,-
FL,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
GL,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
JH,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-


### Part B </font>

In [9]:
reader = Reader(rating_scale=(1, 4))
data = Dataset.load_from_df(ratings[['User', 'Course', 'Rating']], reader)
trainset = data.build_full_trainset()
sim_options = {'name': 'pearson', 'user_based': True}  # compute cosine similarities between items
algo = KNNBasic(sim_options=sim_options)
algo.fit(trainset)
pred1 = algo.predict(str('EN'), str('Python'),  verbose=True)
pred2 = algo.predict(str('EN'), str('Forecast'), verbose=True)

Computing the pearson similarity matrix...
Done computing similarity matrix.
user: EN         item: Python     r_ui = None   est = 3.00   {'actual_k': 1, 'was_impossible': False}
user: EN         item: Forecast   r_ui = None   est = 2.00   {'actual_k': 1, 'was_impossible': False}


<font color='purple'>EN has the highest correlation with LN who has taken additional courses Python & Forecast which EN has not.
Out of these two courses, we check the predicted rating that EN might give to these two courses and
 since Python has an estimated rating = 3 which is greater than the estimated rating of Forecast = 2, we will
    recommend Python to EN. </font>

### Part C </font>

<font color='purple'> Imputing the blanks with zeroes while calculating cosine similarity, as additional zeroes do not have an impact on the calculation as the formula does not consider mean. </font>

In [10]:
cos_sim=cosine_similarity(df.fillna(0))

user_sim_df = pd.DataFrame(cos_sim, index = df.T.columns, columns = df.T.columns)
user_sim_df.round(2)

User,AF,AH,BA,DS,DU,EN,FL,GL,JH,KG,LN,MG,MH,RW,SA
User,,,,,,,,,,,,,,,
AF,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.71,0.00,0.45,1.00
AH,0.00,1.00,0.00,0.00,0.71,0.00,1.00,1.00,0.71,0.00,0.00,0.00,0.62,0.00,0.00
BA,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.71,0.00,0.45,1.00
DS,0.00,0.00,0.00,1.00,0.47,0.88,0.00,0.00,0.47,0.00,0.76,0.00,0.31,0.00,0.00
DU,0.00,0.71,0.00,0.47,1.00,0.37,0.71,0.71,1.00,0.00,0.40,0.00,0.77,0.00,0.00
EN,0.00,0.00,0.00,0.88,0.37,1.00,0.00,0.00,0.37,0.00,0.72,0.00,0.25,0.00,0.00
FL,0.00,1.00,0.00,0.00,0.71,0.00,1.00,1.00,0.71,0.00,0.00,0.00,0.62,0.00,0.00
GL,0.00,1.00,0.00,0.00,0.71,0.00,1.00,1.00,0.71,0.00,0.00,0.00,0.62,0.00,0.00
JH,0.00,0.71,0.00,0.47,1.00,0.37,0.71,0.71,1.00,0.00,0.40,0.00,0.77,0.00,0.00


### Part D </font>

In [11]:
sim_options = {'name': 'cosine', 'user_based': True}  # compute cosine similarities between items
algo = KNNBasic(sim_options=sim_options)
algo.fit(trainset)
pred1 = algo.predict(str('EN'), str('Python'),  verbose=True)
pred2 = algo.predict(str('EN'), str('Forecast'), verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
user: EN         item: Python     r_ui = None   est = 3.50   {'actual_k': 2, 'was_impossible': False}
user: EN         item: Forecast   r_ui = None   est = 2.00   {'actual_k': 1, 'was_impossible': False}


<font color='purple'>EN is closest to DS with a similarity = 0.88. But EN has already taken all the courses that DS has rated. So we consider the next closest neighbour, which is LN with a similarity of 0.72. The courses taken by LN which EN has not taken yet are Python and Forecast. Python has a higher predicted rating = 3.5 compared to Forecast = 2. Therefore, Python should be recommended to EN.</font>

### Part E </font>

<font color='purple'> The correlation formula subtracts the ratings of users with the average rating given by that user which adjusts for users' tendency to rate higher or lower by default. Cosine similarity differs from correlation by not subtracting the means. 
    
   The implication is that for cosine similarity, we can impute the missing values with zero without affecting our results as the mean is not considered. 
    
   But for correlation, if we impute the missing/blank values with zero, it impacts our mean which will be used in the correlation calculations. Therefore, we need a more sensible imputation & often the mean of the actual ratings given by that user is considered a good imputation for blank/missing values while calculating correlations. When the blank values are replaced with the mean rating, the formula happens to adjust in such a way that (mean-mean) will be anyway zero for that imputed entry & therefore does not produce any errorneous correlations for ratings which were not present before. </font>

### Part F : 1 
<font color='purple'>It is possible to calculate correlations only for the course pairs which have been rated by more than one user and those ratings need to differ in magnitude. For example, since R Prog has only received ratings of 4 even though by 3 different users, correlation can not be computed for this course. 

If the goal is to recommend a course to EN, we would like to find the correlations of these 4 courses with all other remaining courses which EN has not yet taken: SQL, DM in R, R Prog and Regression. But since R prog can not be computed for correlations, it would be useful only to find correlations for SQL, DM in R and Regression with other remaining courses.
</font>

### Part F : 2

<font color='purple'>Without calculating the correlations for all course pairs, we can see that: 
    
   1. SQL has been rated by 5 other users, out of which 3 users have also rated Spatial. This is the highest combination of courses being co-rated so we should calculate correlation b/w <b> SQL and Spatial </b>
    
   2. Regression has been rated by only one other user "LN", who has taken Python & Forecast as the other two courses not taken by "EN". Out of these two courses, "LN" rated Python higher than Forecast, so it would be useful to calculate correlation between <b> Regression & Python </b>
   
   3. DM in R is rated only by one other user "DS" who has taken other courses SQL and R Prog which EN has already taken. So it would not help to calculate course pair correlation for DM in R if we have to recommend something to EN.</font>

In [30]:
items = df.T
ratings_item=items.apply(lambda row: row.fillna(row.mean()), axis=1)

df1=ratings_item.T
print(df1['SQL'].corr(df1['Spatial']))
print(df1['Regression'].corr(df1['Python']))

0.6428571428571428
0.5000000000000001


### Part G </font>

In [31]:
ratings_item.T.corr().round(2).fillna('-')

Course,DM in R,Forecast,Hadoop,PA 1,Python,R Prog,Regression,SQL,Spatial
Course,,,,,,,,,
DM in R,1,0,-,0,0,-,0.5,0,0
Forecast,0,1,-,0.19,0.5,-,0.5,-0.19,0
Hadoop,-,-,-,-,-,-,-,-,-
PA 1,0,0.19,-,1,0,-,0,0,0
Python,0,0.5,-,0,1,-,0.5,-0.38,0.19
R Prog,-,-,-,-,-,-,-,-,-
Regression,0.5,0.5,-,0,0.5,-,1,0,0
SQL,0,-0.19,-,0,-0.38,-,0,1,0.64
Spatial,0,0,-,0,0.19,-,0,0.64,1


<font color='purple'>EN has taken these 4 courses: SQL, DM in R, R Prog and Regression. Out of these 4 courses, SQL is a course which has highest correlation (0.64) with another course, which is Spatial which happens to be a course EN has not taken yet. Therefore, we would recommend Spatial to EN. </font>